# `etl_transform`
The input are parquet files (with long format) under the same directory.

## Extract
`pd.read_parquet()` extracts the input 

## Transformation
* transform from long to pivot format
* normalise (either log(x+1) or by rank-normalisation)

## Load
* output the data as csv

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

from set_path import *

In [ ]:
def p_to_z(data):
    q=1-data/2
    z=stats.norm.ppf(data)
    return z
p_to_z=np.vectorize(p_to_z)

In [ ]:
class TRANSFORM:
    def __init__(self,pep,norm_method,output1):
        self.dfpep=pep
        self.norm_method=norm_method
        self.output1=output1
    def read_pep(self):
        self.dfpep=pd.read_parquet(self.dfpep)
    def long_to_pivot(self):
        self.dfpep=self.dfpep.pivot(index='ProbenID', columns='Muster', values='Amplitude')
        self.dfpep.fillna(0,inplace=True)
        
    def pep_normalise(self):
        if self.norm_method=='none':
            pass
        elif self.norm_method=='ln': # log (x+1) transformation
            self.dfpep=np.log(self.dfpep+1)
        elif self.norm_method=='rank': # to z-scores
            self.dfpep = self.dfpep.rank()
            # pct is obtained by deviding rank /len(df)+1, in order to avoid getting '1' as pct
            # pct=1 gives z= infinity 
            # therfore we cannot directly apply self.dfpep.rank(pct=1)
            self.dfpep= self.dfpep/ (len(self.dfpep)+1)
            self.dfpep=pd.DataFrame(data=p_to_z(self.dfpep),index=self.dfpep.index,columns=self.dfpep.columns)
     #   print (self.dfpep.head)
        print (self.dfpep.head())
        print (self.dfpep.shape)
    def check_na(self):
        pass
    
    def export_csv(self):
        self.dfpep.to_csv(self.output1)


In [ ]:
# ALL, from long to pivot
x1=TRANSFORM(pep=f2+'reduced_all/',norm_method='rank',output1=f2+'reduced_all_pivot.csv')
x1.read_pep()
x1.long_to_pivot()
x1.pep_normalise()
x1.export_csv()